### Glue를 활용한 데이터 활용하기

Jupyter notebook을 통해 AWS Glue job을 생성하기 전에 먼저 데이터를 확인해보겠습니다.
저희는 앞선 실습에서 RDS MySQL의 데이터를 크롤링하여 metadata를 만들었고, Amazon S3에 데이터가 들어왔다는 전제하에 업로드 시켜두었습니다. 

### 1. Crawl our sample dataset

예제에서는 아래의 경로에 데이터들이 적재되어 있습니다.

    s3://awskrug-data-닉네임/
    jdbc:mysql://RDS_endpoint:3306/datalab

### 2. GlueContext 설정

필요한 AWS Glue의 라이브러리를 가져와서 단일 GlueContext를 설정합니다. 

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
10,application_1569309920903_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3. AWS Glue Crawler가 식별한 스키마를 확인

앞서 크롤러가 만든 database 및 table을 정의하여 스키마를 확인하겠습니다.
실습에서는 아래 3개의 테이블을 조회해 보겠습니다. 
 - order_info
 - user_event_logs
 - user_info

In [2]:
order_info = glueContext.create_dynamic_frame.from_catalog(database="datalab-rds", table_name="datalab_order_info")
order_info.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- timestamp: timestamp
|-- user_id: string
|-- goods_id: int
|-- shop_id: int
|-- price: int

In [3]:
user_event_log = glueContext.create_dynamic_frame.from_catalog(database="datalab-s3", table_name="user_event_logs_2019")
user_event_log.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- col0: string
|-- col1: string
|-- col2: string
|-- col3: string
|-- col4: long
|-- col5: long
|-- partition_0: string
|-- partition_1: string

In [4]:
user_info = glueContext.create_dynamic_frame.from_catalog(database="datalab-rds", table_name="datalab_user_info")
user_info.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- user_id: string
|-- os: string
|-- age: int


위와 같은 방식으로 각각의 데이터베이스에 존재하는 테이블의 스키마 정보를 확인할 수 있습니다.
이를 보고 JOIN하거나 불필요한 컬럼을 제거하거나,이름을 변경한 뒤 조인하는 등의 행위를 할 수 있습니다. 


### 4. 필터링

원하는 필드만 유지하고 특정 필드는 제거하여 필터링 작업을 하겠습니다. 
예를들면, timestamp 필드를 제거한 뒤, user_id는 id로 변경해보겠습니다.

이때 toDF() 변환은 DynamicFrame을 Spark DataFrame으로 변환하므로 SparkSQL에 이미 존재하는 변환을 적용할 수 있습니다.

In [5]:
user_event_log = user_event_log.drop_fields(['timestamp']).rename_field('user_id', 'id')
user_event_log.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+-------------+-----------+-----------+----+----+
|                col0|                col1|                col2|         col3|partition_0|partition_1|col4|col5|
+--------------------+--------------------+--------------------+-------------+-----------+-----------+----+----+
|2019-09-21 00:00:...|K1d8_t3-QIskaSkrx...|       shops_ranking|app_page_view|         09|         21|null|null|
|2019-09-21 00:00:...|lwFZ77v_ygk0uU40t...|      shops_bookmark|app_page_view|         09|         21|null|null|
|2019-09-21 00:00:...|mR-bO6hC9g-m8ERXM...|goods_search_resu...|app_page_view|         09|         21|null|null|
|2019-09-21 00:00:...|K1d8_t3-QIskaSkrx...|      shops_bookmark|app_page_view|         09|         21|null|null|
|2019-09-21 00:00:...|Yjny5AchUWLiuv4kd...|      shops_bookmark|app_page_view|         09|         21|null|null|
|2019-09-21 00:00:...|LZZ0ktGq6hW685TFA...|            my_goods|app_page_view|         09|      

이러한 방식으로 특정 컬럼을 제외할 수도 있고, 리네임할 수 도 있습니다. 

이제는 datalab_user_info와 Order_info를 user_id로 조인하고, 스키마를 출력하겠습니다.

In [6]:
result = Join.apply(order_info, user_info,'user_id','user_id')
result = result.drop_fields(['.user_id'])

print ("Count: ",result.count())
result.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Count:  868
root
|-- price: int
|-- shop_id: int
|-- age: int
|-- timestamp: timestamp
|-- goods_id: int
|-- os: string
|-- .user_id: string
|-- user_id: string


안에 어떠한 값들이 들어있는지 확인해보겠습니다.

In [7]:
result.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------+---+--------------------+--------+---+--------------------+--------------------+
|price|shop_id|age|           timestamp|goods_id| os|            .user_id|             user_id|
+-----+-------+---+--------------------+--------+---+--------------------+--------------------+
|23500|    102| 20|2018-06-11 02:04:...|    5436|iOS|W-CvDQrB4uXbiGGZ8...|W-CvDQrB4uXbiGGZ8...|
|15000|     41| 25|2018-06-11 14:17:...|    1884|And|fYHpMIbtwo3JQ8nPn...|fYHpMIbtwo3JQ8nPn...|
|19800|     32| 32|2018-06-11 12:21:...|    5884|iOS|RWEDTLLkIUVuQQI7Z...|RWEDTLLkIUVuQQI7Z...|
|11500|     12| 15|2018-06-11 21:36:...|    1922|And|xp2UXd5LsDAq7ib1H...|xp2UXd5LsDAq7ib1H...|
|11000|     22| 21|2018-06-11 18:16:...|    3355|iOS|ZqC_h-amqTdhYvv0V...|ZqC_h-amqTdhYvv0V...|
|52000|    145| 23|2018-06-11 03:54:...|    6369|iOS|MRNFeS0sVveDbr_PG...|MRNFeS0sVveDbr_PG...|
|44900|    139| 23|2018-06-11 02:30:...|    1172|iOS|MRNFeS0sVveDbr_PG...|MRNFeS0sVveDbr_PG...|
|25000|     77| 23|2018-06-11 03:57:...|


### 4. 데이터 쓰기

만들어진 데이터를 특정 저장장치(Amazon S3)에 쓰도록하겠습니다.
실습에서는 Amazon S3에만 데이터를 쓰지만, RDBMS, Redhift 등에도 쓸 수 있습니다.


Amazon S3경로에 조인된 데이터를 parquet으로 변환하여 upload하겠습니다.

In [12]:
import boto3

s3 = boto3.client('s3')
s3.put_object(Bucket="awskrug-data-luke2", Key="output/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '008307E51596F76F', 'HostId': 'KqxRHIt+VMxlh0FZRwaEHTC5IVdC9vcTpaeUSoSmPTLoalzY7X9mpGzUZb7R9E0Haqzj8QwvTX4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'KqxRHIt+VMxlh0FZRwaEHTC5IVdC9vcTpaeUSoSmPTLoalzY7X9mpGzUZb7R9E0Haqzj8QwvTX4=', 'x-amz-request-id': '008307E51596F76F', 'date': 'Fri, 27 Sep 2019 05:10:50 GMT', 'etag': '"d41d8cd98f00b204e9800998ecf8427e"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 1}, 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [ ]:
output_AmazonS3 = "s3://awskrug-data-luke2/output/"

print("Writing ...")
glueContext.write_dynamic_frame.from_options(frame = result, connection_type = "s3", connection_options = {"path": output_AmazonS3}, format = "parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 결론

ETL 작업을 하게 되면 한번에 코딩하기도 어렵고, 시간도 많이 걸릴 것 입니다.
이럴 때 AWS Glue endpoint를 사용하여 jupyter notebook 환경을 이용하여 대화형으로 개발할 수 있습니다.